# MODS Archon FITS header lab

Lab notebook for learning how to deconstruct, fix, and work with azcam/archon-generated FITS headers from the MODS instruments.

In [2]:
import os
import numpy as np

# Set up matplotlib

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, LogLocator, NullFormatter

%matplotlib inline

# astropy packages we need for FITS, time, and coordinates

from astropy.io import fits
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, Angle
from astropy.time import Time

# other bits

import time

## Plot setup

In [3]:
# Aspect ratio

aspect = 2.5

#
# Don't change these unless you really need to (we never have).  They make for good resolution
# and scale for insertion into LaTeX and Word documents
#
# fPage is the horizontal fraction of the page occupied by the figure, default 1.0
#
# scaleFac is the LaTeX includegraphics scaling in units of \textwidth, default 1.0
#

fPage = 1.0
scaleFac = 1.0

# Text width in inches - don't change, this is defined by the print layout for most portrait LaTeX templates

textWidth = 6.0 # inches

# Graphic dimensions depending on bitmap or vector format (draft vs production)

figFmt = 'png'
dpi = 600
plotWidth = dpi*fPage*textWidth
plotHeight = plotWidth/aspect
axisFontSize = 8
labelFontSize = 6
lwidth = 0.5
axisPad = 5
wInches = fPage*textWidth # float(plotWidth)/float(dpi)
hInches = wInches/aspect  # float(plotHeight)/float(dpi)

# LaTeX is used throughout for markup of symbols, Times-Roman serif font

plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Times-Roman'],'weight':'bold','size':'16'})

# Font and line weight defaults for axes

matplotlib.rc('axes',linewidth=lwidth)
matplotlib.rcParams.update({'font.size':axisFontSize})

# axis and label padding

plt.rcParams['xtick.major.pad']=f'{axisPad}'
plt.rcParams['ytick.major.pad']=f'{axisPad}'
plt.rcParams['axes.labelpad'] = f'{axisPad}'

## MODS CCD Image Basics

Read and sniff the headers of a MODS FITS file.  Format is multi-extension FITS with 5 extensions
 * PRIMARY - primary HDU with the full FITS header created by azcam including instrument and telescope when enabled.
 * Q1 - Image HDU for CCD quadrant 1
 * Q2 - Image HDU for CCD quadrant 2
 * Q3 - Image HDU for CCD quadrant 3
 * Q4 - Image HDU for CCD quadrant 4
 * CONPARS - BinTable HDU with the Archon controller status snapshot at start of exposure

### CCD Layout in detector coordinates

<img src="MODS_CCD_Readout.png" width="800">

A full frame is 8288x3088 pixels.  There are 32 columns of overscan in each quadrant, so the total number of pixels in each
image file is 8352x3088. The central reference pixel is (4144,1544) for converting to full detector coordinates.

The 4 quadrants are labeled Q1 through Q4 as shown.  Each is 4176x1544 pixels (4144 active + 32 overscan columns).  Quadrants 1 and 3
are readout toward the left corner amplifiers, quadrants 2 and 4 are readout toward the right corner amplifiers.  

A full-frame unbinned image is readout in about 22 seconds with a 0.5 second shutter delay before readout starts.

### Subframe MODS

Subframe region-of-interest (ROI) readout is done in symmetical windows centered on the reference pixel.  We currently
support three subframe ROI modes:
 * 1024x1024, 512x512 pixels/quadrant
 * 3088x3088, 1544x1544 pixels/quadrant
 * 4096x3088, 2048x1544 pixels/quadrant

Each has 32 columns of overscan per quadrant.

## Convenience functions

### fixDataSec(hdu) - fix DATASEC header info in quadrant HDUs

The coordinates in the `DETSEC` and `CCDSEC` keywords for each quadrant and the `LVTn` coordinate transform
coefficients are incorrect.  If you read the raw MODS FITS image using ds9 and open as "Mosaic IRAF", the
quadrants will not be stitched together correctly on the display (huge gaps) and the cursor will read
the wrong coordinates back.  This fixes the headers in post-processing.  Eventually we should dig deep
into the `azcam` code and fix it there, but not now.


In [4]:
def fixDataSec(hdu):
    # get header parameters we need

    c0=int(hdu[0].header['ref-pix1']) # reference pixel 
    r0=int(hdu[0].header['ref-pix2'])

    nc=hdu[1].header['naxis1'] # total size of a quadrant (all same)
    nr=hdu[1].header['naxis2']

    ncbias = hdu[1].header['ovrscan1'] # bias columns

    # compute corrected DETSEC/CCDSEC and LVTn parameters

    detsec1=f"[{c0-nc+ncbias+1}:{c0},{r0-nr+1}:{r0}]"
    detsec2=f"[{c0+nc+ncbias}:{c0+1},{r0-nr+1}:{r0}]" # flip x
    detsec3=f"[{c0-nc+ncbias+1}:{c0},{r0+1}:{r0+nr}]"
    detsec4=f"[{c0+nc+ncbias}:{c0+1},{r0+1}:{r0+nr}]" # flip x

    q24_LTV1 = c0 + nc - ncbias + 1 
    q34_LTV2 = -r0

    # Rrecompute and correct DETSEC and CCDSEC

    hdu[1].header['detsec'] = (detsec1,'Corrected DETSEC')
    hdu[2].header['detsec'] = (detsec2,'Corrected DETSEC')
    hdu[3].header['detsec'] = (detsec3,'Corrected DETSEC')
    hdu[4].header['detsec'] = (detsec4,'Corrected DETSEC')

    hdu[1].header['ccdsec'] = (detsec1,'Corrected CCDSEC')
    hdu[2].header['ccdsec'] = (detsec2,'Corrected CCDSEC')
    hdu[3].header['ccdsec'] = (detsec3,'Corrected CCDSEC')
    hdu[4].header['ccdsec'] = (detsec4,'Corrected CCDSEC')

    # fix LTVn so the ds9 cursor returns correct physical pixel coords 
    # in Q2,Q3, and Q4 (Q1 is OK)

    hdu[2].header['LTV1'] = (q24_LTV1,'Corrected LTV1')
    hdu[4].header['LTV1'] = (q24_LTV1,'Corrected LTV1')

    hdu[3].header['LTV2'] = (q34_LTV2,'Corrected LTV2')
    hdu[4].header['LTV2'] = (q34_LTV2,'Corrected LTV2')

    # all done

    return

### fixArchonTemps(hdu) - fix CCD and Archon temperatures in the Primary HDU

Reads the CONPARS extension (5) and extracts the CCD, Base, and Archon Backplane temperatures at
start of exposure and puts these into primary HDU header.  This corrects wrong `CCDTEMP` and `BASETEMP`
that comes from `azcam`.

In [5]:
def fixArchonTemps(hdu):
    # Get the Archon controller status snapshot from HDU 5 (bintable)

    t = Table(hdu[5].data)
    archonDict = dict(zip(t['Keyword'].tolist(),t['Value'].tolist()))

    # Which sensor is which depends on the MODS channel

    modsID = hdu[0].header['INSTRUME']
    if modsID in ['MODS1B','MODS1R']:
        ccdSensor = 'MOD10/TEMPB'
        baseSensor = 'MOD10/TEMPA'
    else:
        ccdSensor = 'MOD10/TEMPA'
        baseSensor = 'MOD10/TEMPB'
        
    # fix CCDTEMP and BASETEMP

    try:
        ccdTemp = float(archonDict[ccdSensor])
    except:
        ccdTemp = -999.9 # no-read value
    hdu[0].header['CCDTEMP'] = (ccdTemp,'CCD Detector Temperature [deg C]')

    try:
        baseTemp = float(archonDict[baseSensor])
    except:
        baseTemp = -999.9 # no-read value
    hdu[0].header['BASETEMP'] = (baseTemp,'CCD Mount Base Temperature [deg C]')

    # Extract and store the Archon backplane temperature

    try:
        archonTemp = float(archonDict['BACKPLANE_TEMP'])
    except:
        archonTemp = -999.9 # no-read value
    hdu[0].header['ARCHTEMP'] = (archonTemp,'Archon Controller Backplane Temperature [deg C]')

    return

### fixDateTime(hdu) - fix date/time info in the Primary HDU

Converts the `azcam` style `DATE-OBS` keyword in to LBT Archive compliant ISO8601 format (`azcam` is old school).

Also compute MJD and heliocentric and barycentric JD and add it to the headers as `HJD-OBS` and `BJD-OBS` at start
of exposure useful for MODS observations involving precise timing.

In [6]:
def fixDateTime(hdu):

    # fix DATE-OBS to make it FITS- and LBT Archive format compliant ISO8601

    try:        
        dateObs = hdu[0].header["DATE-OBS"]
        utcObs = hdu[0].header["UTC-OBS"]
        newDateObs = f"{dateObs}T{utcObs}"
        hdu[0].header["DATE-OBS"] = (newDateObs,"UTC Date at start of obs")
            
        # compute modified Julian Date
        
        obsTime = Time(newDateObs,format="isot",scale="utc")
        hdu[0].header["MJD-OBS"] = (obsTime.mjd,"Modified JD at start of obs [UTC]")
            
    except:
        # unlikely...
        pass
        
    # Compute heliocentric and barycentric JD.  HJD is in UTC time scale, but BJD is 
    # in Barycentric Dynamical Time (TDB).  We need site and target coordinate info.
        
    try:
        telRA = hdu[0].header["TELRA"]
        telDec = hdu[0].header["TELDEC"]
   
        obsSite = EarthLocation.of_site("LBT")
        obsTime = Time(newDateObs,format="isot",scale="utc",location=obsSite)
        obsCoord = SkyCoord(telRA,telDec,unit=(u.hourangle,u.deg),frame="icrs")
            
        lttBary = obsTime.light_travel_time(obsCoord,"barycentric")
        lttHelio = obsTime.light_travel_time(obsCoord,"heliocentric")
        obsBary = obsTime.tdb + lttBary
        obsHelio = obsTime.utc + lttHelio            

        hdu[0].header["HJD-OBS"] = (obsHelio.mjd,"Heliocentric MJD at start of obs [UTC]")
        hdu[0].header["BJD-OBS"] = (obsBary.mjd,"Barycentric MJD at start of obs [TDB]")
    except:
        # might be TCS offline so no TELRA/TELDEC, or header corruption
        pass
      
    return

### fixMisc(hdu) - fix miscellanous FITS info in the Primary HDU

Where everything else we need to fix/tweak/add is done. Many of these are for back-compatibility with
the original MODS headers where we need to convert azcam-isms into MODS-isms, or add keywords that
are used by the LBTO Archive, pipelines, etc.

Not everything from old MODS translates to new MODS (and vis-versa), but we do what we can

In [7]:
def fixMisc(hdu):

    # DD returns mount alt/az in sexigesimal degrees, LBT data archive is orthodox about
    # using decimal degrees, but DD only returns decimal angles in radians.  So, we use 
    # astropy.coordinates Angle() to make the conversion. Why they can't handle this natively...

    try:         
        decAlt = Angle(hdu[0].header["TELALT"],unit=u.degree).degree
        decAz = Angle(hdu[0].header["TELAZ"],unit=u.degree).degree
        hdu[0].header["TELALT"] = (decAlt,"Telescope Altitude at start of obs [deg]")
        hdu[0].header["TELAZ"] = (decz,"Telescope Azimuth at start of obs [deg]")
    except:
        # unlikely...
        pass

    # Zenith distance is just 90-altitude, but OK
    
    try:         
        zd = 90.0 - Angle(hdu[0].header["TELALT"],unit=u.degree).degree
        hdu[0].header["ZD"] = (zd,"Zenith distance at start of obs [deg]")
    except:
        hdu[0].header["ZD"] = (-99.99,"Zenith distance at start of obs [deg]")

    # LBTWLINK is returned by the DD as a 1/0 boolean, old MODS translated to Up/Down
    
    try:
        lbtWLink = int(hdu[0].header["LBTWLINK"])
        if (lbtWLink == 1):
            hdu[0].header["LBTWLINK"] = ("Up","LBT Weather Station Link State")
        else:
            hdu[0].header["LBTWLINK"] = ("Down","LBT Weather Station Link State")
    except:
        hdu[0].header["LBTWLINK"] = ("Unknown","LBT Weather Station Link State")

    # azcam does not record a global merged CCD ROI, like the old MODS CCDROI keyword, so we  
    # compute it.  Header 1 (IM1) has the relevant dimensions plus overscan, 0 has ref pix
    
    try:
        nc = hdu[1].header['NAXIS1']
        nr = hdu[1].header['NAXIS2']
        overx = hdu[1].header['OVRSCAN1']
        xref = hdu[0].header['REF-PIX1']
        yref = hdu[0].header['REF-PIX2']
        
        sc = int(xref - nc + overx +1)
        ec = int(xref + nc - overx)
        sr = int(yref - nr + 1)
        er = int(yref + nr)
        hdu[0].header["CCDROI"] = (f"[{sc}:{ec},{sr}:{er}]","CCD subframe ROI coords")
    except:
        # unlikely...
        pass

    # azcam stores binning factors in the CCDSUM keyword as "(nx ny)", convert to old-style
    # MODS CCDXBIN and CCDYBIN. Get from header 1 (IM1 extension), all are the same.
    
    try:
        ccdsum = hdu[1].header["CCDSUM"]
        bits = ccdsum.split(" ")
        hdu[0].header["CCDXBIN"] = (int(bits[0]),"CCD X-axis Binning Factor")
        hdu[0].header["CCDYBIN"] = (int(bits[1]),"CCD Y-axis Binning Factor")
    except:
        pass

    # Other stuff goes here
    
    return

## otmProc(hdu) - subtract overscan bias, trim, and merge quadrants into a single image

Computes and substracts the overscan bias for each quadrant, trims the overscan columns, flips the images as needed,
and merges them into a single 32-bit floating FITS image array.

In [8]:
def otmProc(hdu,biasColOff=2,biasRowOff=2,sigClip=3):
    
    # Q2 and Q4 are flipped along rows

    flipRows = [False,True,False,True]

    # Size of data section w/o column bias

    ncQuad = hdu[1].header['naxis1'] - hdu[1].header['ovrscan1']
    nrQuad = hdu[1].header['naxis2']

    ncImg = 2*ncQuad
    nrImg = 2*nrQuad

    # create an empty array to contian the merged image
    
    mergedImg = np.empty((nrImg,ncImg),dtype=np.float32)

    # starting numpy pixels of each quadrant

    startPix = [(0,0),(0,ncQuad),(nrQuad,0),(nrQuad,ncQuad)]

    # process by quadrant

    quadBias = []
    quadStd = []
    for quad in [1,2,3,4]:
        quadData = hdu[quad].data.astype(np.float32) # convert to 32-bit float for processing
        nc = hdu[quad].header['naxis1'] # number of columns
        nr = hdu[quad].header['naxis2'] # number of rows
        ncbias = hdu[quad].header['ovrscan1'] # number of overscan columns

        # extract bias overscan columns
    
        biasCols = quadData[biasRowOff:-biasRowOff,nc-ncbias+biasColOff:]
        bias1d = np.median(biasCols,axis=1)
        medBias = np.median(bias1d)
        stdBias = np.std(bias1d)
        if (sigClip > 0):
            loCut = medBias - sigClip*stdBias
            hiCut = medBias + sigClip*stdBias
            iClip = np.where((bias1d >= loCut) & (bias1d <= hiCut))
            medBias= np.median(bias1d[iClip])
            stdBias = np.std(bias1d[iClip])                                    
        quadBias.append(medBias)
        quadStd.append(stdBias)

        # subtract the median overscan bias from the quadrant

        imgData = quadData[0:,0:nc-ncbias] - medBias

        # is this quadrant flipped? Unflip it

        if flipRows[quad-1]:
            imgData = np.flip(imgData,axis=1)
        
        # insert the debiased quadrant into the full merged image array
    
        sr = startPix[quad-1][0]
        er = sr + nr
        sc = startPix[quad-1][1]
        ec = sc + nc - ncbias
    
        mergedImg[sr:er,sc:ec] = imgData

    # all done return the image

    return mergedImg, quadBias, quadStd


## Open the image, scan and fix the header

In [ ]:
inFile = "MODS1B/mods1b.20251224.0008.fits"

hdu = fits.open(inFile)
hdu.info()

print(len(hdu[0].header['comment']))

# fix header issues

fixDataSec(hdu)
fixArchonTemps(hdu)
fixDateTime(hdu)
fixMisc(hdu)

# check the fixes

print("\nPrimary FITS Header:\n")
print(repr(hdu[0].header))

hdu.close()

## Quadrant Images

Analysis of coordinates and the bias overscan columns. 

Some questions to address with this section:
 * what is the bias level?

Range is about 950-1050 ADU between CCDs, but very close within a given CCD.  Target was ~1000 ADU

 * how clean is the bias?

Bias is very flat, but initial/end columns and top/bottom rows can sometimes have high values

 * How many initial columns to skip?

2 starting columns **and** 2 rows start/end avoids very high values

 * Is the bias flat or is there a trend with row?

Outside the offsets above, it is very flat with rows, so subtracting a scalar instead of a vector is indicated.

This just plots, no writing of the image.

In [ ]:
inFile = "MODS1B/mods1b.20251224.0008.fits"

# open the file

hdu = fits.open(inFile)

# plotting options

showOverscan = False

# Q2 and Q4 are flipped along rows

flipRows = [False,True,False,True]

# bias column analysis

biasColOff = 2 # skip 2 columns at start of biassec
biasRowOff = 2 # skip 2 rows at start/end of biassec 

# reference pixels in x and y

xref = int(hdu[0].header["ref-pix1"])
yref = int(hdu[0].header["ref-pix2"])

print(f"Reference pixel: {xref},{yref}")

# Size of the merged data w/o overscan columns

ncImg = 2*(hdu[1].header['naxis1'] - hdu[1].header['ovrscan1'])
nrImg = 2*hdu[1].header['naxis2']

print(f"Output size: {ncImg}x{nrImg}")
fullImg = np.empty((nrImg,ncImg),dtype=np.float32)

# quadrant

for quad in [1,2,3,4]:
    
    t0 = time.perf_counter() # start performance timer
    
    quadData = hdu[quad].data.astype(np.float32) # convert to 32-bit float for processing
    
    nc = hdu[quad].header['naxis1'] # number of columns
    nr = hdu[quad].header['naxis2'] # number of rows
    ncbias = hdu[quad].header['ovrscan1'] # number of overscan columns
    print(f"\nImage Q{quad}:")
    print(f"  Format: {nc}x{nr}, {ncbias} overscan columns")
    print(f"  dataType: {quadData.dtype}")
    print(f"  BIASSEC={hdu[quad].header['biassec']}")
    print(f"  DATASEC={hdu[quad].header['datasec']}")
    print(f"  DETSEC={hdu[quad].header['detsec']}")
    print(f"  CCDSEC={hdu[quad].header['ccdsec']}")

    # extract bias overscan columns
    
    biasCols = quadData[biasRowOff:-biasRowOff,nc-ncbias+biasColOff:]
    #print(f"  biasCols shape: {biasCols.shape}")
    medBias = np.median(biasCols)
    meanBias = np.mean(biasCols)
    sigBias = np.std(biasCols)
    minBias = np.min(biasCols)
    maxBias = np.max(biasCols)
    print(f"  Bias: Median={medBias:.2f} +/- {sigBias:.2f}, Min={minBias:.2f} Max={maxBias:.2f} Mean={meanBias:.2f}")
    bias1d = np.median(biasCols,axis=1)
    print(f"        clipped median: {np.median(bias1d):.2f} +/- {np.std(bias1d):.2f} mean: {np.mean(bias1d):.2f}")

    # subtract bias from the data

    imgData = quadData[:,:nc-ncbias-1] - medBias
    # print(f"  imgData data type: {imgData.dtype}")
    
    if flipRows[quad-1]:
        imgData = np.flip(imgData,axis=1)
        print(f"  flipped along rows")
        
    t1 = time.perf_counter()
    biasTime = t1 - t0
    print(f"  debias time: {biasTime:.3f} sec")
    
    # basic image stats
    
    imgMed = np.median(imgData)
    imgStd = np.std(imgData)
    print(f"   Min: {np.min(imgData):.2f}")
    print(f"   Max: {np.max(imgData):.2f}")
    print(f"  Mean: {np.mean(imgData):.2f}, Median: {imgMed:.2f}")
    print(f"   Std: {imgStd:.2f}")

    t2 = time.perf_counter()
    statsTime = t2 - t1
    print(f"  stats time: {statsTime:.3f} sec")
    
    # plot bias subtracted image (or whole image)
    
    fig,(ax1,ax2) = plt.subplots(1,2,figsize=(wInches,hInches),dpi=dpi)
    fig.subplots_adjust(wspace=0.3, hspace=0.0)
    
    if showOverscan:
        ax1.imshow(quadData-medBias, cmap="gray",vmin=imgMed-medBias-imgStd,vmax=imgMed-medBias+imgStd)
    else:
        ax1.imshow(imgData, cmap="gray",vmin=imgMed-imgStd,vmax=imgMed+imgStd)
    ax1.tick_params('both',length=2,width=lwidth,which='major',direction='out',top='on',right='on')
    ax1.tick_params('both',length=1,width=lwidth,which='minor',direction='out',top='on',right='on')
    ax1.set_title(f"Q{quad} Bias-subtracted")
    
    # plot column bias vs. row pixel
    
    ax2.tick_params('both',length=4,width=lwidth,which='major',direction='in',top='on',right='on')
    ax2.tick_params('both',length=2,width=lwidth,which='minor',direction='in',top='on',right='on')
    ax2.plot(bias1d,'.',lw=0.5,ms=1,color='black')
    ax2.set_xlabel("Row pixel")
    ax2.set_ylabel("median bias")
    ax2.set_title(f"Q{quad} Overscan Bias")

    plt.show()

hdu.close()

## Overscan debias, trim, and merge quadrants into as single image

Operations:
 * read in the image
   - fix known header issues
   - compute full image size, create empty 32-bit float array
 * for each quadrant:
   - compute and subtract overscan bias
   - if Q2 or Q4, flip in rows
   - add to the full image
 * add the overscan, trimmed, and merged image to the FITS file
   - create a new HDU

In [ ]:
inFile = "MODS1B/mods1b.20251224.0008.fits"

# open the file
     
t0 = time.perf_counter() # start performance timer
    
hdu = fits.open(inFile)

# Fix header issues

fixDataSec(hdu)
fixArchonTemps(hdu)
fixDateTime(hdu)

# subtract overscan bias, trim, and merge quadrants into a single "otm" image

otmImg,quadBias,quadStd = otmProc(hdu)

# make an image header data unit for the OTM image and append it to the main HDU
# with a copy of the full header from the primary HDU plus header cards for
# the bias overscan values subtracted as a record of the processing.

otmHDU = fits.ImageHDU(data=otmImg,header=hdu[0].header,name="Merged")

for quad in [1,2,3,4]:
    otmHDU.header[f"Q{quad}Bias"] = (quadBias[quad-1],f"Q{quad} median overscan bias [DN]")
    otmHDU.header[f"Q{quad}Std"] = (quadStd[quad-1],f"Q{quad} overscan bias stdev [DN]")

hdu.append(otmHDU)

# write to a new file, old base name with _otm suffix (overscan, trimmed, merged)

baseName = os.path.splitext(inFile)[0]
newFITS = f"{baseName}_otm.fits"
hdu.writeto(newFITS,overwrite=True)

t1 = time.perf_counter()
print(f"Subtract overscan bias, trimmed, merged, and saved: {(t1-t0):.3f} sec")

hdu.close()

# quick stats and display 

imgMed = np.median(otmImg)
imgStd = np.std(otmImg)
loCut = imgMed - 3*imgStd
hiCut = imgMed + 3*imgStd
iClip = np.where((otmImg >= loCut) & (otmImg <= hiCut))
imgMed = np.median(otmImg[iClip])
imgMean = np.mean(otmImg[iClip])
imgStd = np.std(otmImg[iClip])

print(f"Full image: median: {imgMed:.4f} +/- {imgStd:.4f} DN, mean: {imgMean:.4f} DN")

# plot bias subtracted image (or whole image)
    
fig,ax = plt.subplots(figsize=(wInches,hInches),dpi=dpi)

ax.imshow(otmImg, cmap="gray",vmin=imgMed-3*imgStd,vmax=imgMed+3*imgStd)
ax.tick_params('both',length=2,width=lwidth,which='major',direction='out',top='on',right='on')
ax.tick_params('both',length=1,width=lwidth,which='minor',direction='out',top='on',right='on')
ax.set_title(f"Processed, merged image",fontsize=labelFontSize)
    
plt.show()